In [3]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [4]:
token = 'vk1.a.otLsAEXa8C6Y5rzitVBbgEm3qjK5Wy-QEtyiM9lI9amp14EHzoQEmH2sRVi00wv2LYdwo8SOOqO1DAvvX-cHbPH3eo-uhFWgJJyWreD5R7_VGWUf0cKjeZj5n7jRN5a44gyqkerx5h0Ast0F0XlvudPgQ3KsfueFxdpliV8fG-a_TogwzhcGI_M_FQY-ShWu7q4KCZICuKuD8BVrrHg6og'

In [8]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [13]:
vk.messages.send(
    chat_id=1,
    random_id=0,
    message='ЧЧЧЧ'
)

0

In [10]:
# vk_session.method('messages.send', {'chat_id': 1, 'message': 'Wake up ELchin and Igramin!!!', 'random_id': 0})

0

In [11]:
# проверка как много груп общих ващих друзей

0

In [14]:
ads_data = pd.read_csv('./ads_data.csv')

In [15]:
ads_data.head()

,date,time,event,platform,ad_id,client_union_id,campaign_union_id,ad_cost_type,ad_cost,has_video,target_audience_count
0,2019-04-01,2019-04-01 00:00:48,view,android,45061,34734,45061,CPM,200.6,0,1955269
1,2019-04-01,2019-04-01 00:00:48,view,web,121288,121288,121288,CPM,187.4,0,232011
2,2019-04-01,2019-04-01 00:01:03,view,android,102737,102535,102564,CPC,60.7,0,4410
3,2019-04-01,2019-04-01 00:01:03,view,android,107564,106914,107564,CPM,217.3,0,62711
4,2019-04-01,2019-04-01 00:01:09,view,android,4922,37,4400,CPC,60.1,0,1183501


In [22]:
# top-10 ads by ctr
ads_data_by_events = ads_data.groupby(['event','ad_id'], as_index=False)\
    .agg({'time':'count'})\
    .pivot(index='ad_id', columns='event', values='time')\
    .reset_index()

In [25]:
ads_data_by_events_ctr = ads_data_by_events.fillna(0).assign(ctr = ads_data_by_events.click/ads_data_by_events.view)

In [33]:
top_ctr = ads_data_by_events_ctr.query("click > 10 & view>2").sort_values('ctr').tail(10)

In [34]:
top_ctr

event,ad_id,click,view,ctr
114,18525,25.0,409.0,0.061125
93,17396,17.0,276.0,0.061594
132,20374,21.0,328.0,0.064024
128,19946,11.0,163.0,0.067485
319,35953,20.0,285.0,0.070175
359,38224,18.0,253.0,0.071146
703,105604,14.0,169.0,0.082840
838,114959,22.0,259.0,0.084942
621,46639,44.0,253.0,0.173913
787,112583,105767.0,351802.0,0.300644


In [36]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [55]:
# send .csv file to message group
result = json.loads(requests.post(vk.docs.getMessagesUploadServer(
        type='doc',   
        peer_id=2000000001
        )['upload_url'], 
    files={'file': open('./top_ctr_data.csv', 'rb')}).text
    )
jsonAnswer = vk.docs.save(file=result['file'], title='Top Clickthrough rate (CTR)', tags=[])

In [53]:
vk.messages.send(
    peer_id=2000000001, 
    random_id=0, 
    attachment=f"doc{jsonAnswer['doc']['owner_id']}_{jsonAnswer['doc']['id']}"
    )

0

In [88]:
# send image file to message group

uploadurl = vk.photos.getMessagesUploadServer(peer_id = 2000000001)['upload_url']
pfile = requests.post(uploadurl, files = {'photo': open('./image.jpg', 'rb')}).json()
jsonAnswer = vk.photos.saveMessagesPhoto(server = pfile['server'], photo = pfile['photo'], hash = pfile['hash'])[0]


In [89]:
vk.messages.send(
    peer_id=2000000001, 
    random_id=0, 
    message='Хайи Ватан(Диге)',
    attachment="photo{}_{}".format(jsonAnswer["owner_id"], jsonAnswer["id"])
    )

0